In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [2]:
# ライブラリのインポート
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

In [3]:
# データの読み込み
train_df = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')
print("Train Data Head:")
print(train_df.head())
print("Test Data Head:")
print(test_df.head())

Train Data Head:
   id         Brand Material    Size  Compartments Laptop Compartment  \
0   0      Jansport  Leather  Medium           7.0                Yes   
1   1      Jansport   Canvas   Small          10.0                Yes   
2   2  Under Armour  Leather   Small           2.0                Yes   
3   3          Nike    Nylon   Small           8.0                Yes   
4   4        Adidas   Canvas  Medium           1.0                Yes   

  Waterproof      Style  Color  Weight Capacity (kg)      Price  
0         No       Tote  Black             11.611723  112.15875  
1        Yes  Messenger  Green             27.078537   68.88056  
2         No  Messenger    Red             16.643760   39.17320  
3         No  Messenger  Green             12.937220   80.60793  
4        Yes  Messenger  Green             17.749338   86.02312  
Test Data Head:
       id   Brand Material    Size  Compartments Laptop Compartment  \
0  300000    Puma  Leather   Small           2.0             

In [4]:
# 欠損値の確認
print("Missing Values in Train Data:")
print(train_df.isnull().sum())

Missing Values in Train Data:
id                         0
Brand                   9705
Material                8347
Size                    6595
Compartments               0
Laptop Compartment      7444
Waterproof              7050
Style                   7970
Color                   9950
Weight Capacity (kg)     138
Price                      0
dtype: int64


In [5]:
# 数値データとカテゴリカルデータの分離
numeric_features = train_df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = train_df.select_dtypes(include=['object']).columns
print("Numeric Features:", numeric_features)
print("Categorical Features:", categorical_features)

Numeric Features: Index(['id', 'Compartments', 'Weight Capacity (kg)', 'Price'], dtype='object')
Categorical Features: Index(['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color'],
      dtype='object')


In [6]:
# 'Price' 列を numeric_features から除外
numeric_features = numeric_features.drop('Price')

In [7]:
# 数値データの欠損値処理
imputer = SimpleImputer(strategy='mean')
train_df[numeric_features] = imputer.fit_transform(train_df[numeric_features])
test_df[numeric_features] = imputer.transform(test_df[numeric_features])
print("Train Data after Imputation:")
print(train_df.head())

Train Data after Imputation:
    id         Brand Material    Size  Compartments Laptop Compartment  \
0  0.0      Jansport  Leather  Medium           7.0                Yes   
1  1.0      Jansport   Canvas   Small          10.0                Yes   
2  2.0  Under Armour  Leather   Small           2.0                Yes   
3  3.0          Nike    Nylon   Small           8.0                Yes   
4  4.0        Adidas   Canvas  Medium           1.0                Yes   

  Waterproof      Style  Color  Weight Capacity (kg)      Price  
0         No       Tote  Black             11.611723  112.15875  
1        Yes  Messenger  Green             27.078537   68.88056  
2         No  Messenger    Red             16.643760   39.17320  
3         No  Messenger  Green             12.937220   80.60793  
4        Yes  Messenger  Green             17.749338   86.02312  


In [8]:
# カテゴリカルデータのワンホットエンコーディング
train_df = pd.get_dummies(train_df, columns=categorical_features)
test_df = pd.get_dummies(test_df, columns=categorical_features)
print("Train Data after One-Hot Encoding:")
print(train_df.head())

Train Data after One-Hot Encoding:
    id  Compartments  Weight Capacity (kg)      Price  Brand_Adidas  \
0  0.0           7.0             11.611723  112.15875         False   
1  1.0          10.0             27.078537   68.88056         False   
2  2.0           2.0             16.643760   39.17320         False   
3  3.0           8.0             12.937220   80.60793         False   
4  4.0           1.0             17.749338   86.02312          True   

   Brand_Jansport  Brand_Nike  Brand_Puma  Brand_Under Armour  \
0            True       False       False               False   
1            True       False       False               False   
2           False       False       False                True   
3           False        True       False               False   
4           False       False       False               False   

   Material_Canvas  ...  Waterproof_Yes  Style_Backpack  Style_Messenger  \
0            False  ...           False           False            Fals

In [9]:
# train_df と test_df の列を揃える
train_df, test_df = train_df.align(test_df, join='left', axis=1, fill_value=0)
print("Aligned Train Data:")
print(train_df.head())
print("Aligned Test Data:")
print(test_df.head())

Aligned Train Data:
    id  Compartments  Weight Capacity (kg)      Price  Brand_Adidas  \
0  0.0           7.0             11.611723  112.15875         False   
1  1.0          10.0             27.078537   68.88056         False   
2  2.0           2.0             16.643760   39.17320         False   
3  3.0           8.0             12.937220   80.60793         False   
4  4.0           1.0             17.749338   86.02312          True   

   Brand_Jansport  Brand_Nike  Brand_Puma  Brand_Under Armour  \
0            True       False       False               False   
1            True       False       False               False   
2           False       False       False                True   
3           False        True       False               False   
4           False       False       False               False   

   Material_Canvas  ...  Waterproof_Yes  Style_Backpack  Style_Messenger  \
0            False  ...           False           False            False   
1         

In [10]:
# 特徴量とターゲットの分離
X = train_df.drop('Price', axis=1)
y = train_df['Price']

In [11]:
# 訓練データとテストデータの分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training and Validation Data Shapes:")
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

Training and Validation Data Shapes:
(240000, 28) (60000, 28) (240000,) (60000,)


In [12]:
# モデルの構築
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [13]:
# モデルのトレーニング
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [14]:
# 予測の実行
y_pred = model.predict(X_val)

In [15]:
# 評価指標の計算
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 1562.7838038845634


In [16]:
# テストデータに対する予測
test_pred = model.predict(test_df.drop('Price', axis=1, errors='ignore'))

In [17]:
# 結果の保存
submission = pd.DataFrame({'id': test_df['id'], 'Price': test_pred})
submission.to_csv('submission.csv', index=False)

In [18]:
# 出力ファイルの表示
print("First and Last 6 rows of the submission file:")
print(pd.concat([submission.head(6), submission.tail(6)]))

First and Last 6 rows of the submission file:
              id      Price
0       300000.0  87.345029
1       300001.0  87.440523
2       300002.0  97.510579
3       300003.0  83.585873
4       300004.0  76.720012
5       300005.0  87.142592
199994  499994.0  75.330072
199995  499995.0  89.036235
199996  499996.0  82.112442
199997  499997.0  83.490923
199998  499998.0  86.697409
199999  499999.0  73.119162
